# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [20]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [21]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [22]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('c3d1988af67b6b8d996e8874fc6acf776f99900e6318a09a25322977e897f9f9')

In [23]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [24]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

SERPAPI_API_KEY = 'c3d1988af67b6b8d996e8874fc6acf776f99900e6318a09a25322977e897f9f9'

In [26]:
todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [27]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [28]:
OBJECTIVE = "Write a weather report for SF today"

In [29]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [33]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

2: Research current weather conditions in San Francisco
3: Identify key factors to include in the weather report (temperature, precipitation, wind, etc.)
4: Choose a format for the weather report (written, video, infographic, etc.)
5: Create an outline for the weather report, including sections for each key factor
6: Write an attention-grabbing introduction for the weather report
7: Gather and include detailed information for each key factor in the report
8: Use visuals, such as charts or images, to enhance the weather report
9: Proofread and edit the weather report for accuracy and clarity
10: Finalize the weather report and publish it for the target audience.
1: Make a todo list

*****NEXT TASK*****

2: Research current weather conditions in San Francisco


> Entering new AgentExecutor chain...
Thought: I should use the Search function to find information about the current weather in San Francisco
Action: Search
Action Input: "current weather in San Francisco"
O

{'objective': 'Write a weather report for SF today'}

In [35]:

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at designing sustainable urban gardens. "
    "Come up with a detailed todo list for this objective: {objective}. Be sure to consider factors like space usage, water conservation, plant types, and community involvement."
)


In [31]:

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for answering questions about gardening, community projects, and sustainability.",
    ),
    Tool(
        name="UrbanGardeningTool",
        func=lambda input: "Output from UrbanGardeningTool for: " + input,
        description="Useful for obtaining advice on plant selection, layout planning, and sustainable practices."
    ),
]


In [32]:

prefix = """You are an AI planner tasked with designing a sustainable urban garden plan. Consider the following details:
- Incorporate space efficiency.
- Focus on water conservation techniques.
- Choose plants suitable for urban settings.
- Ensure community involvement.
Use the completed tasks as context: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""


In [37]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [ ]:

# # New objective for the AI
OBJECTIVE = "Design a sustainable urban garden plan for a community space."

# Create BabyAGI with the updated tools and objective
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

# Execute with the new objective
baby_agi({"objective": OBJECTIVE})
